In [1]:
!module load R/3.5.1

In [126]:
## Installations for py2.7 environment for DelimitR
#conda install notebook -c conda-forge
#conda install mpi4py -c conda-forge
#conda install -c r rpy2
## ------------------------------------------------------------ 
## Must Load Rpy2 to use R through the python notebook
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [30]:
## Should only need to run this once; devtools installed fine, delimitR eventually installed fine

#%%R
#install.packages('devtools', repos = "http://cran.r-project.org")
#devtools::install('delimitR', dependencies = TRUE)

In [127]:
%%R
library(abcrf)
library(devtools)
library(delimitR)
library(randomForest)
library(neuralnet)

## need to actually install these
library(foreach)
library(doParallel)

In [129]:
%%R
getwd()

[1] "/mnt/lfs2/ruff6699/Thujaplicata_AllDataAnalysis/delimitR/Dataset2"


In [132]:
%%R
setwd("/mnt/lfs2/ruff6699/Thujaplicata_AllDataAnalysis/delimitR/Dataset8")

In [133]:
%%R
getwd()

[1] "/mnt/lfs2/ruff6699/Thujaplicata_AllDataAnalysis/delimitR/Dataset8"


In [135]:
%%R
## Set up files and Parameters for DelimitR analysis

## observed SFS file
observedSFS <- 'jSFS_dataset8_noMono'

## file designating which sample with which population
traitsfile <- 'traits_d8.txt'

## 2 populations
observedtree <- '(0,1);'

## Allow migration between the two populations
migmatrix <- matrix(c(FALSE, TRUE, 
                    TRUE, FALSE),
                    nrow = 2, ncol = 2, byrow = TRUE)

## Allow either of these scenerios? Yes
divwgeneflow <- TRUE
seccontact <- TRUE

## Max number of migration events (can only be 1 if two populations)
maxedges <- 1

## 2 "species"
obsspecies<- 2

## number of alleles from Coast and Inland
obssamplesize <- c(42,42)

## roughly, the number of SNPS. will change to a range of SNPS in future/manually
obssnps <- 1041

## Prefix for files of this analysis
obsprefix <- 'Thuja'

In [97]:
%%R
## Setting up prior ranges on parameter values
## Some priors will need to be set up manually as well

## Population size prior for both populations
obspopsizeprior <- list(c(100000,1000000),c(100000,1000000))
 
## if gen length is 10 yrs for thuja, this divergence time prior is 500,000 years to 10 million years
obsdivtimeprior <- list(c(50000,1000000))

## if gen length is 10 yrs for thuja, this divergence time prior is 1,000 years to 50,000 years
#obsdivtimeprior <- list(c(100,5000))

## migration rate prior can remain the same
obsmigrateprior <- list(c(0.0000005,0.00005))

In [69]:
%%R
##Don't ever really need to re-run because set up most of the models manually

## this function generates all of the tpl and est files!! 
setup_fsc2(tree=observedtree,
           nspec=obsspecies,
           samplesizes=obssamplesize,
           nsnps=obssnps,
           prefix=obsprefix,
           migmatrix=migmatrix,
           popsizeprior=obspopsizeprior,
           divtimeprior=obsdivtimeprior,
           migrateprior=obsmigrateprior,
           secondarycontact= seccontact,
           divwgeneflow= divwgeneflow,
           maxmigrations = maxedges)

## For the Record; this creats 4 .tpl and 4 .est files.
## Thuja_1: 2 population model of immediate coalescence by populations. Panmixia
## Thuja_2: 2 population model with ancient coalescence event. Ancient Vicariance

## needed to adjust these models to have assymmetric gene flow, so just added an additional migration parameter 
##        with a different name
## Thuja_3: 2 population model with secondary contact: Secondary contact
## Thuja_4: 2 population model of divergence with gene flow: Divergence with gene flow

## Needed to create a few more models, so made these ones below manually and added them to file

## Have a feeling these two will not be identifiable (Yeah they're not 12/18/2018, so combine them into 1 model 
##    where migration can range from 0 to 0.00005)
## Thuja_5: 2 popoulation model with a very recent coalescence event and migration starting at 0. Recent Dispersal
## CUT Thuja_cut: 2 population model with a very recent coalescence event, no migration.

## Still need to think about which models to add growth rates to, when in the models to add them

In [136]:
%%R
## define a parallel version of the function fastsimcoalsims (named fastsimcoalsims_Par) ##
###########################################################################################

fastsimcoalsims_Par <- function (prefix, pathtofsc, nreps)
{   listoftpl <- list()
    listofest <- list()
    tpllist <- system(paste("ls ", prefix, "*.tpl", sep = ""),
                        intern = T)
    estlist <- system(paste("ls ", prefix, "*.est", sep = ""),
                        intern = T)
    listoftpl <- c(listoftpl, tpllist)
    listofest <- c(listofest, estlist)
    foreach(j=1:length(listoftpl)) %dopar% {
        print(paste(pathtofsc, " -t ", prefix, "_", j, ".tpl",
                " -e ", prefix, "_", j, ".est", " -n 1 --msfs -q --multiSFS -x -E",
                nreps, sep = ""))
        system(paste(pathtofsc, " -t ", prefix, "_", j, ".tpl",
                " -e ", prefix, "_", j, ".est", " -n 1 --msfs -q --multiSFS -x -E",
                nreps, sep = ""), ignore.stdout = TRUE)
    }
}
## Users must register the number of cores to use with doParallel
ncores=11
registerDoParallel(cores=ncores)



In [137]:
%%R
getwd()

[1] "/mnt/lfs2/ruff6699/Thujaplicata_AllDataAnalysis/delimitR/Dataset8"


In [138]:
%%R
## Simulate the data using fastsimcoal2
fastsimcoalsims_Par(prefix=obsprefix,
                pathtofsc='../fsc26',
                nreps=10000)


[[1]]
[1] 0

[[2]]
[1] 0

[[3]]
[1] 0

[[4]]
[1] 0

[[5]]
[1] 0

[[6]]
[1] 0

[[7]]
[1] 0

[[8]]
[1] 0

[[9]]
[1] 0

[[10]]
[1] 0

[[11]]
[1] 0



In [20]:
%%R
## Also now is an opportune moment to clean our working directory of eronious file
## DO NOT DO THIS until all params files are save so you know the params you simulated under
#clean_working(prefix=obsprefix)

In [139]:
%%R
traitsfile

[1] "traits_d8.txt"


In [140]:
%%R
## define the number of class to bin SFS by and assemble the prior from the simulated data
nclasses <- 5


FullPrior <- makeprior(prefix=obsprefix,
                       nspec=obsspecies,
                       nclasses=nclasses,
                       getwd(),
                       traitsfile = traitsfile,
                       threshold=100, 
                       thefolder = 'Prior',
                       ncores = 11)

In [141]:
%%R
getwd()

[1] "/mnt/lfs2/ruff6699/Thujaplicata_AllDataAnalysis/delimitR/Dataset8"


In [142]:
%%R
## Have a look at the prior
#FullPrior
save(FullPrior, file="FullPrior_dataset8.Rdata")

In [10]:
%%R
#load(file="FullPrior_dataset11.Rdata")

In [143]:
%%R
##  We want to remove rows that have zero variance because these bins add nothing to the analysis
ReducedPrior <- Prior_reduced(FullPrior)
ReducedPrior 

         V1  V2  V3  V4    V5  V6  V7 V8   V9 V11 V12 V13 V16  V17   V21
1       707  46   0   0   0.0  38 122 40  0.0   3  33  52   0  0.0   0.0
2       732  40   1   0   0.0  50 102 32  0.5   2  34  47   0  0.5   0.0
3       732  48   1   0   0.0  41 108 33  0.0   0  29  49   0  0.0   0.0
4       725  37   2   0   0.0  46 116 33  1.5   1  26  53   0  0.5   0.0
5       735  31   2   0   0.0  46 114 29  0.0   1  38  45   0  0.0   0.0
6       715  36   3   0   0.0  45 110 36  1.0   0  37  58   0  0.0   0.0
7       732  46   0   0   0.0  37 100 37  1.0   0  33  54   0  1.0   0.0
8       733  45   0   0   0.0  38 122 33  1.0   0  28  41   0  0.0   0.0
9       696  47   2   0   0.0  39 127 40  0.0   1  36  53   0  0.0   0.0
10      764  44   0   0   0.0  34  99 33  0.5   1  29  35   0  1.5   0.0
11      714  53   3   0   0.0  40  97 40  0.5   1  33  59   0  0.5   0.0
12      707  46   1   0   0.0  47 110 26  0.0   0  55  49   0  0.0   0.0
13      729  45   1   0   0.0  42 102 31  0.5   1  

In [56]:
%%R
## Also now is an opportune moment to clean our working directory of eronious file
## DO NOT DO THIS until all params files are save so you know the params you simulated under
#clean_working(prefix=obsprefix)

In [27]:
%%R
Models <- as.factor(FullPrior[,'Model'])
length(Models)

sample <- as.integer(runif(10000, 1, length(Models)))
Trainingdata <-  data.frame(Models, ReducedPrior)
Testdata <- data.frame(Models[sample], ReducedPrior[sample,])
colnames(Testdata) <- colnames(Trainingdata)
Trainingdata <- Trainingdata[-sample,]
head(Testdata)

            Models  V1 V2 V3 V4 V5 V6  V7 V8 V9 V11 V12 V13 V16 V17 V21 V22
74380 Thuja_8_MSFS 641 36  1  0  0 51 142 29  0   1  41  44   0   0   0   0
49261 Thuja_5_MSFS  73 10  7  0  1 23   0  0  0   8   0   0  12   0 852   0
59727 Thuja_6_MSFS 250 42 28 20 12  8   0  0  0   9   0   0   3   0 614   0
82284 Thuja_9_MSFS 659 35  1  0  0 50 132 28  1   0  28  52   0   0   0   0
45368 Thuja_5_MSFS 138 10  3  3  4  7   0  0  0   4   0   0   4   0 813   0
51232 Thuja_6_MSFS 167 12  3 10  2 28   0  0  0  13   0   0  13   0 738   0


In [28]:
%%R
Trainingdata

              Models  V1  V2  V3    V4    V5  V6  V7   V8   V9 V11 V12 V13 V16
3       Thuja_1_MSFS 660  28   1   0.0   0.0  36 134 26.0  0.0   0  33  67   0
4       Thuja_1_MSFS 642  34   0   0.0   0.0  41 145 32.0  0.0   0  31  59   0
5       Thuja_1_MSFS 645  28   0   0.0   0.0  59 142 28.0  0.0   0  35  49   0
6       Thuja_1_MSFS 612  36   0   0.0   0.0  57 149 39.0  0.0   0  41  52   0
8       Thuja_1_MSFS 642  38   1   0.0   0.0  48 145 29.0  0.0   0  41  42   0
9       Thuja_1_MSFS 669  26   0   0.0   0.0  57 126 25.5  0.0   0  34  48   0
10      Thuja_1_MSFS 664  30   0   0.0   0.0  45 140 29.0  1.0   1  28  47   0
11      Thuja_1_MSFS 640  24   0   0.0   0.0  44 152 33.0  0.0   0  40  53   0
12      Thuja_1_MSFS 641  32   0   0.0   0.0  54 143 24.0  0.0   0  40  52   0
13      Thuja_1_MSFS 624  41   0   0.0   0.0  63 146 27.0  0.0   0  31  54   0
14      Thuja_1_MSFS 649  22   0   0.0   0.0  48 147 33.0  0.0   0  33  53   0
15      Thuja_1_MSFS 670  31   0   0.0   0.0  53 131

In [42]:
%%R
train.subset <- Trainingdata[as.integer(runif(10000,1,110000)),]
na.rm(train.subset)
head(train.subset)
#myNN <- neuralnet(Models~., data = train.subset, hidden=3)


Error in na.rm(train.subset) : could not find function "na.rm"


/mnt/ceph/ruff6699/miniconda3/envs/py2.7/lib/python2.7/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: Error in na.rm(train.subset) : could not find function "na.rm"

  warnings.warn(x, RRuntimeWarning)


In [33]:
%%R
myNN$result.matrix

NULL


In [35]:
%%R
nn.results <- compute(myNN, Testdata)


Error in cbind(1, pred) %*% weights[[num_hidden_layers + 1]] : 
  requires numeric/complex matrix/vector arguments


/mnt/ceph/ruff6699/miniconda3/envs/py2.7/lib/python2.7/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: Error in cbind(1, pred) %*% weights[[num_hidden_layers + 1]] : 
  requires numeric/complex matrix/vector arguments

  warnings.warn(x, RRuntimeWarning)


In [144]:
%%R
Models <- as.factor(FullPrior[,'Model'])
Trainingdata <- data.frame(Models, ReducedPrior)
myRF.group <- abcrf(Models~., data = Trainingdata, ntree=1000, paral = TRUE, group=list("Thuja_1_MSFS",
                        "Thuja_2_MSFS", "Thuja_3_MSFS", "Thuja_4_MSFS", "Thuja_5_MSFS", "Thuja_6_MSFS", 
                        "Thuja_7_MSFS", c("Thuja_8_MSFS","Thuja_9_MSFS","Thuja_10_MSFS","Thuja_11_MSFS")))

In [145]:
%%R
myRF.group


Call:
 abcrf(formula = Models ~ ., data = Trainingdata, group = list("Thuja_1_MSFS", "Thuja_2_MSFS", "Thuja_3_MSFS", "Thuja_4_MSFS", "Thuja_5_MSFS", "Thuja_6_MSFS", "Thuja_7_MSFS", c("Thuja_8_MSFS", "Thuja_9_MSFS", "Thuja_10_MSFS", "Thuja_11_MSFS")), ntree = 1000, paral = TRUE) 
includes the axes of a preliminary LDA

Number of simulations: 110000
Out-of-bag prior error rate: 19.2309%

Confusion matrix:
     g1   g2   g3   g4   g5   g6   g7    g8 class.error
g1 6587    0    0    0    0    0    0  3413    0.341300
g2    0 6389   62 2926    0  252  368     3    0.361100
g3    0    5 7095  208    0    0  347  2345    0.290500
g4    0 1934  251 7199    0   60  546    10    0.280100
g5    0    0    0    0 9067  933    0     0    0.093300
g6    0  429    0  223  986 8290   72     0    0.171000
g7    0   48  329  498    2   45 9066    12    0.093400
g8 2813    1 1977   23    0    0   33 35153    0.121175


In [146]:
%%R
myRF.group$model.rf$confusion.matrix
write.table(myRF.group$model.rf$confusion.matrix, file="Dataset8_RFconfusionMatrix_GROUP.csv", sep="\t")

In [147]:
%%R
## Construct a Random Forest classifier using reduced prior and look at error rates
myRF <- RF_build_abcrf(ReducedPrior,FullPrior,1000)
myRF

Growing trees.. Progress: 51%. Estimated remaining time: 29 seconds.

Call:
 abcrf(formula = Models ~ ., data = Trainingdata, ntree = ntrees, paral = TRUE) 
includes the axes of a preliminary LDA

Number of simulations: 110000
Out-of-bag prior error rate: 33.8727%

Confusion matrix:
              Thuja_1_MSFS Thuja_2_MSFS Thuja_3_MSFS Thuja_4_MSFS Thuja_5_MSFS
Thuja_1_MSFS          8005            0            2            0            0
Thuja_2_MSFS             0         6396           62         2926            0
Thuja_3_MSFS             5            5         7900          212            0
Thuja_4_MSFS             0         1947          246         7197            0
Thuja_5_MSFS             0            0            0            0         9042
Thuja_6_MSFS             0          419            0          225          972
Thuja_7_MSFS             0           46          320          515            2
Thuja_8_MSFS          2037            0          947            0            0
Thuja

In [113]:
%%R
myRF$model.rf$confusion.matrix
#save(myRF, file="myRFobject_P42_S2000.Rdat")
##Looks like models 2 and 4 are gettig confused; which are AV and AV with migration
## and models 3 and 5 which are Secondary Contact and Recent Dispersal

              Thuja_1_MSFS Thuja_2_MSFS Thuja_3_MSFS Thuja_4_MSFS Thuja_5_MSFS
Thuja_1_MSFS          8485            0            0            0            0
Thuja_2_MSFS             0         7974           44         1698            0
Thuja_3_MSFS             0           19         8687          122            0
Thuja_4_MSFS             0         1201          228         8343            0
Thuja_5_MSFS             0            0            0            0         9007
Thuja_6_MSFS             0          162            0          183          832
Thuja_7_MSFS             0           27          270           67            0
Thuja_8_MSFS          1861            0          516            0            0
Thuja_9_MSFS          2251            0          314            0            0
Thuja_10_MSFS           48            0          253            3            0
Thuja_11_MSFS           50            0          324            2            0
              Thuja_6_MSFS Thuja_7_MSFS Thuja_8_MSFS

In [148]:
%%R
attributes(myRF)
#myRF$prior.err
attributes(myRF$model.rf)
#plot(myRF, training=FullPrior, pdf=TRUE)
write.table(myRF$model.rf$confusion.matrix, file="Dataset8_RFconfusionMatrix.csv", sep="\t")

In [23]:
%%R
#err <- err.abcrf(myRF, training=FullPrior)
#plot(err)

NULL


In [50]:
%%R
##classic RF and not ABC_RF
#classic_rf <- randomForest(Model ~ ., data=FullPrior, ntree = 500 )
#classic_rf

##pretty much perform the same

NULL


In [151]:
%%R
##prep obderved obs file
nclasses <- 5
observedSFS <- 'jSFS_dataset8_noMono'
traitsfile <- 'traits_d8.txt'

myobserved <- prepobserved(
  observedSFS,
  FullPrior,
  ReducedPrior,
  nclasses,
  obsspecies,
  traitsfile=traitsfile,
  threshold = 100)

In [ ]:
##run this in the terminal
#!python binSFS.py

In [152]:
%%R
myobserved

          V1       V2        V3        V4           V5       V6       V7
1   745.7113 32.74797 12.283277 0.7139312 1.717877e+00 138.1220 34.89805
2   749.3922 34.63670 10.653716 1.3798278 1.713354e+00 143.9188 35.61425
3   749.9404 32.82030  9.214389 2.3867175 1.710969e+00 139.8619 33.23685
4   747.4103 34.81163 11.931116 0.4522347 7.178766e-01 146.1972 39.06583
5   757.1174 31.59146 10.596653 1.8899693 1.000070e+00 145.3440 31.06764
6   755.5288 30.85231 10.174068 1.4069901 1.710994e+00 147.4749 33.14374
7   752.5598 32.87218 11.604808 0.3900114 1.717876e+00 138.1901 39.13494
8   757.5116 32.23030  8.631113 2.3371262 1.720263e+00 141.0618 34.44868
9   753.6402 33.72664  5.557411 2.1894039 1.710998e+00 141.1193 40.11173
10  761.3143 28.73065  9.844967 1.1502773 1.710997e+00 141.3014 33.49966
11  764.8897 30.58561 10.145675 2.1273100 1.710994e+00 132.8807 33.19658
12  755.9396 35.35235  8.481276 2.3361131 1.002426e+00 141.3753 34.45891
13  745.0038 36.77980 12.719975 0.7875432 7.178759e

In [153]:
%%R
prediction<-c()

In [ ]:
##### So will need to make all of the obs sfs fit into the binned file...which means i need them to all be moved into 
#### the processed form and then ind. to the binned form, to then all be in one file as binned

## step 1. make individual subsampled SFS output from isaac - into processed file
## step 2. take processed file and run through SFS_binned_pop2.py script
## step 3. have all binned files write output to one binned file
## step 4. make cross prediction

In [154]:
%%R
##make prediction on data with ABC_RF from delimitR
prediction <- RF_predict_abcrf(myRF, myobserved[1:100,], ReducedPrior, FullPrior, 500)
prediction


    selected model votes model1 votes model2 votes model3 votes model4
1     Thuja_7_MSFS            0           80          202          210
2     Thuja_7_MSFS            0           66          250          192
3     Thuja_7_MSFS            0           70          222          191
4     Thuja_7_MSFS            0           88          233          198
5     Thuja_7_MSFS            0           62          227          176
6     Thuja_7_MSFS            2           54          243          183
7     Thuja_7_MSFS            0           89          227          194
8     Thuja_7_MSFS            0           65          227          203
9     Thuja_7_MSFS            1           68          260          179
10    Thuja_7_MSFS            1           51          244          184
11    Thuja_7_MSFS            1           56          223          190
12    Thuja_7_MSFS            0           63          242          190
13    Thuja_7_MSFS            0           80          231          215
14    

In [155]:
%%R
as.matrix(prediction)

       selected model votes model1 votes model2 votes model3 votes model4
  [1,]              7            0           80          202          210
  [2,]              7            0           66          250          192
  [3,]              7            0           70          222          191
  [4,]              7            0           88          233          198
  [5,]              7            0           62          227          176
  [6,]              7            2           54          243          183
  [7,]              7            0           89          227          194
  [8,]              7            0           65          227          203
  [9,]              7            1           68          260          179
 [10,]              7            1           51          244          184
 [11,]              7            1           56          223          190
 [12,]              7            0           63          242          190
 [13,]              7            0    

In [156]:
%%R
write.table(as.matrix(prediction), sep="\t", file="PredictionOut_Dataset8.csv")

In [157]:
%%R
prediction <- RF_predict_abcrf(myRF.group, myobserved[1:100,], ReducedPrior, FullPrior, 500)
prediction

    selected group votes group1 votes group2 votes group3 votes group4
1               g7            0           76          258          201
2               g7            2           65          321          162
3               g7            1           60          292          165
4               g7            2           73          319          171
5               g7            1           58          272          168
6               g7            2           52          313          154
7               g7            2           75          296          181
8               g7            1           58          291          175
9               g7            1           67          305          163
10              g7            3           49          327          146
11              g7            1           56          295          167
12              g7            1           60          293          160
13              g7            2           79          280          199
14    

In [159]:
%%R
write.table(as.matrix(prediction), sep="\t", file="PredictionOutGroup_Dataset8.csv")